# Dunham's Data

## Setting up

In [1]:
from geopy.distance import geodesic
# import networkx as nx
import pandas as pd
# import seaborn as sns

In [2]:
# pd.set_option('display.max_rows', 999)

In [3]:
# sns.set()

In [4]:
# %matplotlib inline

## Loading data

In [5]:
col_used = [
    'DATE', 'CITY1', 'COUNTRY1', 'CITY2', 'COUNTRY2', 'WORKING', 'HOTEL_ADDRESS',
    'VENUE1', 'VENUE_TYPE1', 'VENUE2', 'VENUE_TYPE2', 'NOTES', 'SOURCE',
]
dunham_df = pd.read_excel('data/src/DunhamsData_EverydayItinerary_DatasetSample.xlsx',
                          usecols='A:G,I:N', names=col_used)
dunham_df.fillna('', inplace=True)

In [6]:
dunham_df.shape

(10, 13)

In [7]:
dunham_df.head()

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
0,1947-03-14,New York City,USA,,,y,,Hotel Delmonico,n,Roxy,n,"Diary--""All Star Show for the Wounded"" between...",Display Ad New York Times (1923-Current file);...
1,1947-04-07,New York City,USA,,,y,,,,,,Diary--rehearsal,SIU Box 142 Folder 7
2,1947-06-28,Mexico City,Mexico,,,y,"Arenal #7, San Angel, Mexico, D.F.",,,,,,
3,1947-07-01,Mexico City,Mexico,,,y,"Arenal #7, San Angel, Mexico, D.F.",,,,,Rehearsing every day at Dunham's rented house;...,SIU Box 10 Folder 4
4,1947-07-06,Mexico City,Mexico,,,y,"Arenal #7, San Angel, Mexico, D.F.",,,,,,


In [8]:
dunham_df.tail(1)

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
9,1948-01-04,San Francisco,USA,,,y,,Geary Theater,c,,,,Bal Negre program


## Pre-processing data

In [9]:
dunham_df.DATE = dunham_df.DATE.astype(str)
for col in dunham_df.columns[1:]:
    dunham_df[col] = dunham_df[col].apply(str.strip)

In [10]:
def clean_city(x, y):
    if x == 'Hamilton':
        if y == 'Canada':
            r = 'Hamilton (CA)'
        else: # y == 'New Zealand'
            r = 'Hamilton (NZ)'
#     elif x == 'in transit':
#         r = ''
    else:
        r = x
    return r

dunham_df.CITY1 = dunham_df.apply(lambda row: clean_city(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.CITY2 = dunham_df.apply(lambda row: clean_city(row.CITY2, row.COUNTRY2), axis=1)

In [11]:
def clean_country(city, country):
    x = country
    y = city
    if y == 'Gibraltar':
        r = 'Gibraltar'
    elif x == 'Newfoundland':
        r = 'Canada'
    else:
        r = x
    return r

dunham_df.COUNTRY1 = dunham_df.apply(lambda row: clean_country(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.COUNTRY2 = dunham_df.apply(lambda row: clean_country(row.CITY2, row.COUNTRY2), axis=1)

In [12]:
# def clean_working(x):
#     if x.startswith('[[') and x.endswith(']]'):
#         r = ''
#     else:
#         r = x
#     return r

# dunham_df.WORKING = dunham_df.WORKING.apply(clean_working)

In [13]:
def clean_hotel_address(x):
    r = x # do not clean for now
    return r

dunham_df.HOTEL_ADDRESS = dunham_df.HOTEL_ADDRESS.apply(clean_hotel_address)

In [14]:
# def clean_performance_yn(x):
#     if x == '?':
#         r = ''
#     elif x == 'N':
#         r = 'n'
#     else:
#         r = x
#     return r

# dunham_df.PERFORMANCE_YN = dunham_df.PERFORMANCE_YN.apply(clean_performance_yn)

In [15]:
# def clean_theater1(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER1 = dunham_df.THEATER1.apply(clean_theater1)

In [16]:
# def clean_theater2(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER2 = dunham_df.THEATER2.apply(clean_theater2)

In [17]:
# def clean_mode_transit(x):
#     if x == '[[chartered bus Reno to Los Angeles?]]':
#         r = 'bus'
#     elif x == 'drive':
#         r = 'car'
#     elif x == 'drive?':
#         r = 'car'
#     elif x.startswith('plane'):
#         r = 'plane'
#     elif x == 'train; boat train':
#         r = 'train, boat, train'
#     elif x == 'train; plane; boat':
#         r = 'train, plane, boat'
#     else:
#         r = x
#     return r

# dunham_df.MODE_TRANSIT = dunham_df.MODE_TRANSIT.apply(clean_mode_transit)

In [18]:
# def clean_transit_cities(x):
#     if x == 'Geneva; Rome':
#         r = 'Geneva (Switzerland), Rome (Italy)'
#     elif x == 'transit to Turin':
#         r = ''
#     elif x == 'transit through London':
#         r = 'London (England)'
#     elif x == 'transit: PanAm 201 flies NYC Laguardia - San Juan - Port of Spain - Belem':
#         r = 'San Juan (Puerto Rico), Port of Spain (Trinidad and Tobago), Belem (Brazil)'
#     elif x == 'Los Cerrillos airport':
#         r = ''
#     elif x == 'transit':
#         r = ''
#     elif x == 'destination':
#         r = ''
#     elif x == 'transit via Boston':
#         r = 'Boston (USA)'
#     elif x == 'in transit':
#         r = ''
#     elif x == 'last performance':
#         r = ''
#     elif x == 'London (Liverpool station), England; Harwich (Harwich Quay), England':
#         r = 'Harwich (England)'
#     elif x == 'New York':
#         r = ''
#     elif x == 'Chicago':
#         r = 'Chicago (USA)'
#     elif x == 'Gander, Newfoundland, Canada':
#         r = 'Gander (Canada)'
#     elif x == 'Zurich, Switzerland':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Zurich':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Rome, Italy; Geneva, Switzerland':
#         r = 'Rome (Italy), Geneva (Switzerland)'
#     elif x == 'Vevey, Switzerland':
#         r = 'Vevey (Switzerland)'
#     elif x == 'Amsterdam, Netherlands':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Furet de P*?': # ???
#         r = ''
#     elif x == 'San Diego; Tijuana':
#         r = 'San Diego (USA), Tijuana (Mexico)'
#     elif x == 'Portworth':
#         r = 'Fort Worth (USA)'
#     elif x == 'Miami':
#         r = 'Miami (USA)'
#     elif x == 'Los Angeles':
#         r = 'Los Angeles (USA)'
#     elif x == "El Paso, TX----I think probably this was to go to Mexico, and I don't think she ever went to Mexico":
#         r = 'El Paso (USA)'
#     elif x == 'Honolulu':
#         r = 'Honolulu (Hawaii)'
#     elif x == 'Coolangatta':
#         r = 'Coolangatta (Australia)'
#     elif x == '[[Dunham Brisbane to Perth via Sydney and Melbourne--date open: https://osu.app.box.com/file/325139873147]]':
#         r = ''
#     elif x == 'Malacca':
#         r = 'Malacca (Malaysia)'
#     elif x == 'Amsterdam':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Lyon; Bordeaux; Bayonne':
#         r = 'Lyons (France), Bordeaux (France), Bayonne (France)'
#     elif x == 'Irun, Spain':
#         r = 'Irun (Spain)'
#     elif x == 'Irun, Spain; Hendaye, France':
#         r = 'Irun (Spain), Hendaye (France)'
#     elif x == 'Marseilles':
#         r = 'Marseille (France)'
#     elif x == 'Piraeus, Greece':
#         r = 'Piraeus (Greece)'
#     elif x == 'Port Said, Egypt':
#         r = 'Port Said (Egypt)'
#     elif x == 'Munich':
#         r = 'Munich (Germany)'
#     elif x == 'Schwarzbach; Ravensberg':
#         r = 'Schwarzbach (Germany), Ravensberg (Germany)'
#     elif x == 'Colmar; Ligny en Barois; Rosay en Brie':
#         r = 'Colmar (France), Ligny-en-Barrois (France), Rozay-en-Brie (France)'
#     elif x == 'Verneson; Valence':
#         r = 'Vernaison (France), Valence (France)'
#     elif x == 'Alicante':
#         r = 'Alicante (Spain)'
#     elif x == 'Granada':
#         r = 'Granada (Spain)'
#     elif x == 'La Linea, Spain':
#         r = 'La Linea (Spain)'
#     else:
#         r = x
#     return r

# dunham_df.TRANSIT_CITIES = dunham_df.TRANSIT_CITIES.apply(clean_transit_cities)

In [19]:
def clean_notes(x):
    r = x # do not clean for now
    return r

dunham_df.NOTES = dunham_df.NOTES.apply(clean_notes)

In [20]:
def clean_source(x):
    r = x # do not clean for now
    return r

dunham_df.SOURCE = dunham_df.SOURCE.apply(clean_source)

In [21]:
dunham_df.fillna('', inplace=True)

## Processing data

In [22]:
geolocations = { # (lat, lon)
    'Mexico City': (19.433333, -99.133333),
    'New York City': (40.7127, -74.0059),
    'San Diego': (32.715, -117.1625),
    'San Francisco': (37.783333, -122.416667),
    'Stockton, CA': (37.975556, -121.300833),
}

### Grouping locations

In [23]:
distances = set()
cities = set(dunham_df.CITY1.unique()).union(dunham_df.CITY2.unique())
cities.discard('')
cities.discard('in transit')
for i, city1 in enumerate(cities):
    p1 = geolocations[city1]
    for j, city2 in enumerate(cities):
        if j > i: # and city1 != city2:
            p2 = geolocations[city2]
            dst = geodesic(p1, p2).km
            distances.add((city1, city2, dst))

In [24]:
len(distances)

10

In [25]:
sorted(distances, key=lambda x: x[2])[:10]

[('Stockton, CA', 'San Francisco', 100.45777277568092),
 ('Stockton, CA', 'San Diego', 694.1804200964181),
 ('San Francisco', 'San Diego', 737.8514518146319),
 ('Mexico City', 'San Diego', 2321.4317013516998),
 ('Stockton, CA', 'Mexico City', 2970.451620820282),
 ('San Francisco', 'Mexico City', 3036.6053907108394),
 ('New York City', 'Mexico City', 3357.6419513494357),
 ('New York City', 'San Diego', 3915.5011469250653),
 ('New York City', 'Stockton, CA', 4039.0194401772533),
 ('New York City', 'San Francisco', 4138.60863656365)]

In [26]:
THRESHOLD = 15

In [27]:
grouped_cities_dct = {
    'Petion-Ville': 'Port-au-Prince',
    'Los Angeles': 'Los Angeles area',
    'Hollywood': 'Los Angeles area',
    'Beverly Hills': 'Los Angeles area',
    'Fiesole': 'Florence',
#     'Mosman': 'Sydney',
}

In [28]:
for k in grouped_cities_dct:
    dunham_df.replace(k, grouped_cities_dct[k], inplace=True)

In [29]:
dunham_df.shape

(10, 13)

## Saving data

In [30]:
dunham_df.to_csv('data/out/dunham_1947-60.csv', index=False)